# Tornado warnings
In this script, we match tornado watches and warnings to tornado events.  See the report and comments below for details of this procedure.  The warnings data totals about 15GB, so we do not include it here.

In [3]:
import shapefile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display, Latex
import glob

In [4]:
# Load warnings data for 1986-2014
# Watches were only recorded 2006+
#warnings_fn = '2014_all\wwa_201401010000_201412312359'
warnings_dir = r'C:\Users\Owner\Desktop\am207_all_warnings\\'

sf_readers = [shapefile.Reader(glob.glob(sub_dir + r'\*')[0].split('.')[0]) for sub_dir in glob.glob(warnings_dir + '\????_all')]

# Load the warning data into pandas
wds = [pd.DataFrame(data=sf.records(), columns=[f[0] for f in sf.fields[1:]]) for sf in sf_readers]
wd = pd.concat(wds, join='inner')

# Select only tornadoes
wd = wd[wd.PHENOM == 'TO']
print 'Number of watches/warnings: {}'.format(len(wd))

# Remove entries without a county code
wd= wd[[d.strip() != '' for d in wd.NWS_UGC]]
print 'Number of with a NWS_UGC code: {}'.format(len(wd))

# Extract state abbrev
wd['state'] = [d[:2] for d in wd['NWS_UGC']]

# Extract County or Zone type
wd['CZ_code'] = [d[2] for d in wd['NWS_UGC']]

# Remove Zones (harder mapping to counties)
wd = wd[wd.CZ_code == 'C']
print 'Number of with a county code: {}'.format(len(wd))

# Extract FIPS county ID
wd['FIPS'] = [int(d[3:]) for d in wd['NWS_UGC']]

# Parse datetimes
wd.ISSUED = pd.to_datetime(wd.ISSUED, utc=True)
wd.EXPIRED = pd.to_datetime(wd.EXPIRED, utc=True)
#wd.INIT_ISS = pd.to_datetime(wd.INIT_ISS, utc=True)  # Not all years have this, and we don't use it
#wd.INIT_EXP = pd.to_datetime(wd.INIT_EXP, utc=True)

# Convert some fields to numeric values
wd.ETN = wd.ETN.astype(int)
wd.AREA_KM2 = wd.AREA_KM2.astype(float)

# Preview data
wd.head()

Number of watches/warnings: 265130
Number of with a NWS_UGC code: 263257
Number of with a county code: 224565


,WFO,ISSUED,EXPIRED,PHENOM,GTYPE,SIG,ETN,STATUS,NWS_UGC,AREA_KM2,state,CZ_code,FIPS
8,FWD,1986-02-03 03:45:00,1986-02-03 04:15:00,TO,C,W,1,   ,TXC367,2379.70,TX,C,367
38,BMX,1986-02-04 23:30:00,1986-02-05 00:30:00,TO,C,W,1,   ,ALC063,1762.87,AL,C,63
54,FWD,1986-02-05 21:22:00,1986-02-05 22:15:00,TO,C,W,2,   ,TXC159,752.05,TX,C,159
55,FWD,1986-02-05 21:22:00,1986-02-05 22:15:00,TO,C,W,2,   ,TXC223,2022.04,TX,C,223
58,FWD,1986-02-05 21:55:00,1986-02-05 23:00:00,TO,C,W,3,   ,TXC449,1072.31,TX,C,449


In [5]:
# Read tornado data
td_fn = '1950-2014_torn.csv'
td = pd.read_csv(td_fn, names=['om', 'yr', 'mo', 'dy', 'date', 'time', 'tz', 'st', 'stf', 'stn', 'f', 'inj', 'fat', 'loss', 'closs', 'slat', 'slon', 'elat', 'elon', 'length', 'wid', 'ns', 'sn', 'sg', 'f1', 'f2', 'f3', 'f4'])

print 'Number of tornadoes: {}'.format(len(td))

# Construct a pandas datetime
td['datetime_gmt'] = td.date + td.time
td.datetime_gmt = pd.to_datetime(td.datetime_gmt, format='%Y-%m-%d%H:%M:%S', errors='raise', exact=True)
# This is the timezone conversion from CST to UTC.  Do we have to worry about daylight savings time?
td.loc[td.tz == 3,'datetime_gmt'] += pd.Timedelta('6h')
# We include tornadoes with an unknown timezone because the chances of them producing a false match are neglegible
print '{} tornadoes with unknown timezone'.format(((td['tz'] != 3) & (td['tz'] != 9)).sum())

# Remove redundant time fields (all tz == 3)
#del td['yr'], td['mo'], td['dy'], td['date'], td['time'], td['tz']

td = td.reset_index()

# Preview data
td.head()

Number of tornadoes: 59945
36 tornadoes with unknown timezone


,index,om,yr,mo,dy,date,time,tz,st,stf,...,length,wid,ns,sn,sg,f1,f2,f3,f4,datetime_gmt
0,0,1,1950,1,3,1950-01-03,11:00:00,3,MO,29,...,9.5,150,2,0,1,0,0,0,0,1950-01-03 17:00:00
1,1,1,1950,1,3,1950-01-03,11:00:00,3,MO,29,...,6.2,150,2,1,2,189,0,0,0,1950-01-03 17:00:00
2,2,1,1950,1,3,1950-01-03,11:10:00,3,IL,17,...,3.3,100,2,1,2,119,0,0,0,1950-01-03 17:10:00
3,3,2,1950,1,3,1950-01-03,11:55:00,3,IL,17,...,3.6,130,1,1,1,135,0,0,0,1950-01-03 17:55:00
4,4,3,1950,1,3,1950-01-03,16:00:00,3,OH,39,...,0.1,10,1,1,1,161,0,0,0,1950-01-03 22:00:00


Now add the following columns to the tornado data:
- datetime_gmt: a numpy datetime64 object that encode the date and time of the touchdown.  Actually added above.
- has_warning: 1 if the tornado had a warning issued, 0 otherwise
- has_watch: 1 if the tornado had a watch issued, 0 otherwise
- warning_time: the number of minutes between the issuance of the first warning and the tornado touchdown. Negative = warning issued after touchdown.

We consider a watch/warning to be issued for a given tornado if the tornado passes through a county that had an active warning at the time.

In [7]:
# For each county a tornado has passed through, see if that county ever had a warning at any time
merged = pd.merge(td, wd, right_on=('FIPS','state'), left_on=('f1','st'))
merged = merged.append(pd.merge(td, wd, right_on=('FIPS','state'), left_on=('f2','st')))
merged = merged.append(pd.merge(td, wd, right_on=('FIPS','state'), left_on=('f3','st')))
merged = merged.append(pd.merge(td, wd, right_on=('FIPS','state'), left_on=('f4','st')))
merged = merged.set_index('index')
print 'Watches/warnings matched to a tornado, regardless of time: {}'.format(len(merged))
print 'Warnings matched to a tornado, regardless of time: {}'.format((merged.SIG == 'W').sum())

# For each warning matched to a tornado's county, see if it was during the right time
# We only have a single timestamp for a given tornado (presumably touchdown time)
# We let a warning be issued up to ~2 hours after touchdown (because tornadoes can last several hours),
# and let it end up to ~1 hour before touchdown.
# This is useful because sometimes new warnings are issued instead of existing ones being extended, but we want to count
# the original warning as the first warning for warning_time purposes
merged = merged[(merged.ISSUED - pd.Timedelta('2h') <= merged.datetime_gmt) & (merged.datetime_gmt <= merged.EXPIRED + pd.Timedelta('1h'))]
print 'Watches/warnings matched to a tornado, including time: {}'.format(len(merged))

# There are some duplicates, which can happen because a tornado can leave then re-enter a county
#merged = merged.drop_duplicates()
print 'Warnings matched to a tornado, including time: {}'.format((merged.SIG == 'W').sum())
#display(merged.head())

# Group watches/warnings by tornado
grouped = merged.groupby(level=0)

# Add the new columns
td_new = td.copy()
td_new['has_warning'] = False
td_new['has_watch'] = False
td_new['warning_time'] = 0
td_new.ix[grouped.grouper.levels[0], 'has_warning'] = grouped['SIG'].apply(lambda x: bool((x == 'W').sum()))
td_new.ix[grouped.grouper.levels[0], 'has_watch'] = grouped['SIG'].apply(lambda x: bool((x == 'A').sum()))

# We can have multiple warnings for a single row,
# so we need to find the one with the largest warning time
just_warnings = merged[merged.SIG == 'W']
minutes = ((just_warnings.datetime_gmt - just_warnings.ISSUED)/np.timedelta64(1, 'm'))
minutes = minutes.groupby(level=0)
td_new.ix[minutes.grouper.levels[0], 'warning_time'] = minutes.max()
print 'Number of tornadoes with a warning: {}'.format(len(minutes))

# Remove unnecessary column
del td_new['index']

print 'Number with watch and warning: {}'.format((td_new.has_watch & td_new.has_warning).sum())

#Preview data
display(td_new.head())

Watches/warnings matched to a tornado, regardless of time: 6620017
Warnings matched to a tornado, regardless of time: 3493362
Watches/warnings matched to a tornado, including time: 60180
Warnings matched to a tornado, including time: 51292
Number of tornadoes with a warning: 24444
Number with watch and warning: 6064


,om,yr,mo,dy,date,time,tz,st,stf,stn,...,sn,sg,f1,f2,f3,f4,datetime_gmt,has_warning,has_watch,warning_time
0,1,1950,1,3,1950-01-03,11:00:00,3,MO,29,1,...,0,1,0,0,0,0,1950-01-03 17:00:00,False,False,0
1,1,1950,1,3,1950-01-03,11:00:00,3,MO,29,1,...,1,2,189,0,0,0,1950-01-03 17:00:00,False,False,0
2,1,1950,1,3,1950-01-03,11:10:00,3,IL,17,1,...,1,2,119,0,0,0,1950-01-03 17:10:00,False,False,0
3,2,1950,1,3,1950-01-03,11:55:00,3,IL,17,2,...,1,1,135,0,0,0,1950-01-03 17:55:00,False,False,0
4,3,1950,1,3,1950-01-03,16:00:00,3,OH,39,1,...,1,1,161,0,0,0,1950-01-03 22:00:00,False,False,0


In [10]:
# Write out to file
td_new_fn = td_fn.split('.')
td_new_fn.insert(-1,'warnings')
td_new_fn = '.'.join(td_new_fn)
td_new.to_csv(td_new_fn,header=False,index=False)
print 'Wrote csv to {}'.format(td_new_fn)

Wrote csv to 1950-2014_torn.warnings.csv
